# 範例

實作模糊與邊緣檢測

- 透過 Gaussian Filter 實作模糊操作
- 透過 Sobel Filter 實作邊緣檢測

In [2]:
import cv2
import numpy as np

img = cv2.imread('lena.png')

## Filter 概念

* filter 又稱做 kernel，概念上是一個固定大小的矩陣，掃過圖片經過計算後取得新的圖片矩陣
* 假設以一個 3\*3 的 filter 操作，每次運算都會把 9 個 pixel 的值相乘相加得到一個值，可以把這個操作想像為特徵提取


**把 Filter 掃過圖片時我們要做兩個矩陣的運算**

目前根據計算順序的不同主要分為 Cross-Correlation 跟 Convolution

* cross-correlation 內積
* convolution 相反的內基

### Filter 操作 - Padding

根據前面介紹的操作都是把 n 個值經過計算得到 1 個值，這樣圖片會變小，所以我們通常會在圖片周圍加上額外的值，確保運算完之後跟原圖大小一樣。但因為操作關係，周圍填甚麼值會影響最後結果。這邊根據不同情況常見的操作有以下幾種：

* 補零
* 補鄰近 pixel 值
* 補整張圖片 pixel 值的 mean
* 鏡射

### Filter 操作: 模糊

最簡單的概念是只要把周遭的 pixel 全部平均就好

但比較常使用的是 Gaussian Filter，認為中心點的資訊還是最重要的

* Gaussian Blur 有時候也會有去雜訊的效果

不是convolution，例如3\*3 的filter對於3\*3的操作是乘以$\dfrac{1}{9}\begin{bmatrix} 1 & 1& 1\\ 1 & 1& 1\\1 & 1& 1\\\end{bmatrix}$

### Filter 操作: 邊緣檢測 - Sobel

邊緣的特性是兩側的顏色差別很大

跟模糊不一樣的地方是邊緣檢測是要加強邊緣特性

* 通常會用 Gray Scale 圖來做邊緣檢測
* 基本的邊緣檢測：Sobel filter

## 模糊

In [3]:
img_blur = img.copy()

# 重複多次 Gaussian 模糊的操作來加深模糊的程度
img_blur = cv2.GaussianBlur(img_blur, (5, 5), 0) #third para is sigmaX: X axis s.d. set 0 s.t. by filter (5,5)
img_blur = cv2.GaussianBlur(img_blur, (5, 5), 0)
img_blur = cv2.GaussianBlur(img_blur, (5, 5), 0)

#　組合 + 顯示圖片
img_show = np.hstack((img, img_blur))
while True:
    cv2.imshow('Gaussian Blur', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## 邊緣檢測

### 組合 x-axis, y-axis 的影像合成

In [8]:
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 對 x 方向做 Sobel 邊緣檢測
img_sobel_x = cv2.Sobel(img_grey, cv2.CV_16S, dx=1, dy=0, ksize=3)
# cv2.Sobel(img_grey, cv2.CV_16S 因為有負數所以用int16, dx=1求導檢測差異大處=邊緣, dy=0, ksize=3)
img_sobel_x = cv2.convertScaleAbs(img_sobel_x) #變成非負整數

# 對 y 方向做 Sobel 邊緣檢測
img_sobel_y = cv2.Sobel(img_grey, cv2.CV_16S, dx=0, dy=1, ksize=3)
img_sobel_y = cv2.convertScaleAbs(img_sobel_y)

# x, y 方向的邊緣檢測後的圖各以一半的全重進行合成
img_sobel_combine = cv2.addWeighted(img_sobel_x, 0.5, img_sobel_y, 0.5, 0) #0 加到最後結果的值

# x,y 方向同時是邊緣
img_sobel_xy = cv2.Sobel(img_grey, cv2.CV_16S, dx=1, dy=1, ksize=3)
img_sobel_xy = cv2.convertScaleAbs(img_sobel_xy)


#　組合 + 顯示圖片
img_show = np.hstack((img_sobel_x, img_sobel_y, img_sobel_combine, img_sobel_xy))
while True:
    cv2.imshow('Edge Detection', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break